# DINOv2

In [2]:
import torch
from torchvision import datasets, transforms


# Load DINOv2 onto GPU
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)
model.eval()

# Optimal CIFAR-10 transform (zero-padding preserves original 32x32 content)
transform = transforms.Compose([
    transforms.Pad((96, 96)),  # (224-32)/2 = 96 pixels padding
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset with pinned memory (faster GPU transfer)
cifar = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

loader = torch.utils.data.DataLoader(
    cifar,
    batch_size=512,  # Max batch size that fits in 16GB GPU
    num_workers=4,
    pin_memory=True  # Faster data transfer to GPU
)

# Extraction with automatic GPU-CPU handling
embeddings, labels = [], []
with torch.no_grad():
    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        embeddings.append(model(images).cpu())  # Async copy to CPU
        labels.append(targets)

embeddings = torch.cat(embeddings)  # Shape: [10000, 384]
labels = torch.cat(labels)

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:01<00:00, 73.6MB/s]
100%|██████████| 170M/170M [00:28<00:00, 5.90MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: